In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os

mingw_bin = "C:/msys64/mingw64/bin"
os.add_dll_directory(mingw_bin)
import ctypes
lin = ctypes.cdll.LoadLibrary("./cmake-build-debug/liblinear_model.dll")

lin.create_linear_model.argtypes = [ctypes.c_int32]
lin.create_linear_model.restype = ctypes.c_void_p

lin.predict_linear_model.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float)]
lin.predict_linear_model.restype = ctypes.c_float

lin.release_linear_model.argtypes = [ctypes.c_void_p]
lin.release_linear_model.restype = None

lin.train_linear_model.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float), ctypes.c_int32, ctypes.c_float, ctypes.c_int32]
lin.train_linear_model.restype = None

lin.create_ovo_classifier.argtypes = [ctypes.c_int32]
lin.create_ovo_classifier.restype = ctypes.c_void_p

lin.train_ovo_classifier.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_int32), ctypes.c_int32, ctypes.c_int32, ctypes.c_float]
lin.train_ovo_classifier.restype = None

lin.predict_ovo_classifier.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float)]
lin.predict_ovo_classifier.restype = ctypes.c_int32

lin.release_ovo_classifier.argtypes = [ctypes.c_void_p]
lin.release_ovo_classifier.restype = None


mlp = ctypes.cdll.LoadLibrary("./cmake-build-debug/libmlp.dll") 

mlp.create_mlp_model.argtypes = [ctypes.POINTER(ctypes.c_int), ctypes.c_int]
mlp.create_mlp_model.restype = ctypes.c_void_p

mlp.train_mlp_model.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int,
    ctypes.c_float,
    ctypes.c_int,
    ctypes.c_bool,
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int
]
mlp.predict_mlp_model.restype = ctypes.POINTER(ctypes.c_float)

mlp.predict_mlp_model.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_bool
]

mlp.release_mlp_model.argtypes = [ctypes.c_void_p]

LOG_FUNC_TYPE = ctypes.CFUNCTYPE(None, ctypes.c_char_p)

@LOG_FUNC_TYPE
def logger_callback(msg):
    print(msg.decode())

In [2]:
classnames_array = ['buffalo', 'elephant', 'zebre']
size = 32

### Changement de résolution des images du dataset:

In [3]:
for classname in classnames_array:
    folder_path = 'images/' + classname 
    for filename in os.listdir(folder_path):
        img = Image.open(folder_path + '/' + filename).convert("RGB")
        img = img.resize(size=(size,size))
        new_folder = './images' + str(size) + '/' + classname
        if not os.path.exists(new_folder):
            os.makedirs(new_folder)
        img.save(new_folder + '/' + filename, format="JPEG")

### Organisation du Dataset pour l'entrainement

In [ ]:
data = []
folder_path = './images32/'
for classname in os.listdir(folder_path):  
    for filename in os.listdir(folder_path + classname):
        img = Image.open(folder_path + classname + '/' + filename)
        index = classname.index(classname)
        one_hot = np.zeros(len(classnames_array))
        one_hot[index] = 1.0
        img_array = np.array(img) / 255.0
        data.append([one_hot, img_array])
data = np.array(data, dtype=object)



### Test avec MLP

In [19]:
layer_sizes = [size*size*3, 256, 64, 32, 16, len(classnames_array)]
layer_array = (ctypes.c_int * len(layer_sizes))(*layer_sizes)
model = mlp.create_mlp_model(layer_array, len(layer_sizes) - 1)
mlp.set_logger(logger_callback)

np.random.shuffle(data)
split_index = int(len(data) * 0.8)
data_train = data[:split_index]
data_test = data[split_index:]
X_train = np.array([item[1] for item in data_train], dtype=np.float32)
Y_train = np.array([item[0] for item in data_train], dtype=np.float32)
X_test = np.array([item[1] for item in data_test], dtype=np.float32)
Y_test = np.array([item[0] for item in data_test], dtype=np.float32)



X_train_reshaped = X_train.reshape((X_train.shape[0], -1))  # (N, 3072)
X_train_c = X_train_reshaped.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_train_c = Y_train.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
X_test_reshaped = X_test.reshape((X_test.shape[0], -1))  # (N, 3072)
X_test_c = X_test_reshaped.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_test_c = Y_test.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
epochs = 5000
learning_rate = 0.01
batch_size = 32


mlp.train_mlp_model(model, X_train_c, Y_train_c, epochs, learning_rate, batch_size, True, X_test_c, Y_test_c, len(data_test))

[C++ LOG] Epoch 500: Accuracy = 86.50%
[C++ LOG] Epoch 1000: Accuracy = 93.36%
[C++ LOG] Epoch 1500: Accuracy = 95.13%
[C++ LOG] Epoch 2000: Accuracy = 95.80%
[C++ LOG] Epoch 2500: Accuracy = 95.80%
[C++ LOG] Epoch 3000: Accuracy = 95.80%
[C++ LOG] Epoch 3500: Accuracy = 95.80%
[C++ LOG] Epoch 4000: Accuracy = 95.80%
[C++ LOG] Epoch 4500: Accuracy = 95.80%
[C++ LOG] Epoch 5000: Accuracy = 96.02%


1

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

X_train_reshaped = X_train.reshape((X_train.shape[0], -1))
X_test_reshaped = X_test.reshape((X_test.shape[0], -1))
Y_train_cat = to_categorical(Y_train, num_classes=len(classnames_array))
Y_test_cat = to_categorical(Y_test, num_classes=len(classnames_array))

def ensure_categorical(Y, num_classes):
    Y = np.array(Y)
    if len(Y.shape) == 1:
        return to_categorical(Y, num_classes)
    elif Y.shape[1] == num_classes:
        return Y 
    else:
        raise ValueError(f"Format inattendu pour Y (shape: {Y.shape})")

Y_train_cat = ensure_categorical(Y_train, len(classnames_array))
Y_test_cat = ensure_categorical(Y_test, len(classnames_array))

model = Sequential([
    Input(shape=(X_train_reshaped.shape[1],)),
    Dense(256, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(len(classnames_array), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_reshaped, Y_train_cat,
          epochs=200,
          batch_size=32,
          validation_data=(X_test_reshaped, Y_test_cat),
          verbose=2)

predictions = model.predict(X_test_reshaped)
pred_labels = np.argmax(predictions, axis=1)
acc = accuracy_score(Y_test, pred_labels)
print(f"\nTest Accuracy: {acc * 100:.2f}%")

Epoch 1/200
29/29 - 1s - 34ms/step - accuracy: 0.3193 - loss: 5.5564 - val_accuracy: 0.3348 - val_loss: 1.6179
Epoch 2/200
29/29 - 0s - 6ms/step - accuracy: 0.3613 - loss: 1.1640 - val_accuracy: 0.4846 - val_loss: 1.0505
Epoch 3/200
29/29 - 0s - 7ms/step - accuracy: 0.4464 - loss: 1.0819 - val_accuracy: 0.3833 - val_loss: 1.1824
Epoch 4/200
29/29 - 0s - 7ms/step - accuracy: 0.4188 - loss: 1.0675 - val_accuracy: 0.4361 - val_loss: 1.0875
Epoch 5/200
29/29 - 0s - 7ms/step - accuracy: 0.4718 - loss: 1.0517 - val_accuracy: 0.5110 - val_loss: 0.9998
Epoch 6/200
29/29 - 0s - 7ms/step - accuracy: 0.5083 - loss: 0.9950 - val_accuracy: 0.4097 - val_loss: 1.0961
Epoch 7/200
29/29 - 0s - 7ms/step - accuracy: 0.4343 - loss: 1.0592 - val_accuracy: 0.4846 - val_loss: 0.9952
Epoch 8/200
29/29 - 0s - 7ms/step - accuracy: 0.5481 - loss: 0.9802 - val_accuracy: 0.4581 - val_loss: 1.0064
Epoch 9/200
29/29 - 0s - 7ms/step - accuracy: 0.5525 - loss: 0.9607 - val_accuracy: 0.5727 - val_loss: 0.9750
Epoch 10/

ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass targets

In [18]:
mlp.release_mlp_model(model)

1

In [21]:
img = Image.open('C:/Users/lgrdp/Downloads/images (1).jpeg').convert("RGB")
img = img.resize(size=(32,32))
img_data = np.array(img) / 255.0

output_array = mlp.predict_mlp_model(model, img_data.ctypes.data_as(ctypes.POINTER(ctypes.c_float)), True)
output_array = ctypes.cast(output_array, ctypes.POINTER(ctypes.c_float * layer_sizes[-1])).contents
output = list(output_array)
class_index = 0
for i in range(len(output)):
    if output[i] == max(output):
        class_index = i
print('predicted class : ' + classnames_array[class_index])
print(output)


predicted class : buffalo
[0.999660313129425, -0.0020802810322493315, -0.43730825185775757]
